In [ ]:
from model import get_new_model, run_validation
import torch
import matplotlib.pyplot as plt
from monai.data import DataLoader
from data import get_dataset
from loss import get_loss_func
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
model = get_new_model()
weights = torch.load("final_weights (copy).pt")
model.load_state_dict(weights)

In [ ]:
_, dataset = get_dataset("/home/espenbfo/datasets/segmentation/Task07_Pancreas/dataset.json", 1234567)
dataloader = DataLoader(dataset,batch_size=1, shuffle=False)



In [ ]:
it = iter(iter(dataloader))

In [ ]:
batch = next(it)

preds = run_validation(model, batch["image"]).detach()
preds = torch.softmax(preds, dim=1)


In [ ]:
import numpy as np
MIN_LABEL_SMOOTHING = 1e-5
MAX_LABEL_SMOOTHING = 1e3

smoothing = MIN_LABEL_SMOOTHING + (MAX_LABEL_SMOOTHING-MIN_LABEL_SMOOTHING)*(1+np.cos(4/100*np.pi))/2
print(f"{smoothing:.2g}")

In [ ]:
length = preds.shape[0]
labels = [0,1,2]
length = len(labels)
print(preds.min())
print(preds.max())
print("0:",(batch["label"]==0).sum())
print("1:",( batch["label"]==1).sum())
print("2:", (batch["label"]==2).sum())
loss_func = get_loss_func(loss_smoothing=1e3)
fig,axes = plt.subplots(length, 3, figsize=(6, 2*length))

slic = 16


for i in range(length):
    prediction = preds[0][i, :,:,slic]
    image = batch["image"][0][0, :,:,slic]
    label = batch["label"][0][0, :,:,slic] == i
    print((batch["label"]==i).sum())
    loss = loss_func(preds, batch["label"]).detach()
    print(f"{i}, {prediction.max():.2f}, {prediction.sum():.2f}, {loss:.2f}")
    axes[i][0].imshow(image)
    axes[i][1].imshow(label)
    axes[i][2].imshow(prediction)

In [ ]:
import cv2

def make_mp4_from_slices(slices, filename):
    slices = np.moveaxis(np.array(slices), 0, 2)
    slices*=255
    slices = slices.astype(np.uint8)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    print(slices.shape[:2])
    writer = cv2.VideoWriter(filename, fourcc, 5.0, (slices.shape[:2]))

    for i in range(slices.shape[3]):
        writer.write(slices[:,:,:, i])
    writer.release()
    

In [ ]:
make_mp4_from_slices(preds[0], "50epoch.mp4")